In [1]:
// Import Apollo Client core components via esm.sh for Deno compatibility
import { ApolloClient, InMemoryCache, HttpLink } from 'https://esm.sh/@apollo/client/core?pin=v3.9.9';
import { gql } from 'https://esm.sh/@apollo/client/core?pin=v3.9.9';
import * as Plot from "https://esm.sh/@observablehq/plot";
import { document } from "jsr:@manzt/jupyter-helper";

// Define the GraphQL endpoint
const graphqlUri = 'https://incidentdatabase.ai/api/graphql';

// Create the Apollo Client instance
const client = new ApolloClient({
  link: new HttpLink({ uri: graphqlUri }),
  cache: new InMemoryCache(),
  // It's often helpful to disable caching in notebook environments for fresh data
  defaultOptions: {
    watchQuery: { fetchPolicy: 'no-cache' },
    query: { fetchPolicy: 'no-cache' },
  },
});

In [2]:
// Import gql tag (if not already imported)
// import { gql } from '[https://esm.sh/@apollo/client/core?pin=v3.9.9';](https://esm.sh/@apollo/client/core?pin=v3.9.9';)

const GET_ALL_CSET_CLASSIFICATIONS = gql`
  query GetAllCSETClassifications {
    classifications(
      filter: { namespace: { EQ: "CSETv1" } }

    ) {
      attributes {
        short_name
        value_json
      }
      incidents {
        incident_id
      }
    }
  }
`;

// --- Fetching (same structure as before, just uses the new query) ---
console.log("Fetching all CSET classification data...");

const { data: {classifications : allClassificationData}, error } = await client.query({
    query: GET_ALL_CSET_CLASSIFICATIONS,
});    


Fetching all CSET classification data...


In [3]:
// === Processing Cell (Simplified) ===

// Assume 'allClassificationData' exists from the previous cell (result of the GraphQL query).
// Assume 'fetchAllClassError' also exists and is null if the fetch was successful.

let heatmapData = []; // Initialize with empty array as default


console.log(`Processing ${allClassificationData.length} CSET classification items...`);

// Helper function to safely parse value_json
const parseValueJson = (value_json) => {
    // Treat null, undefined, or empty string as "Unknown"
    if (!value_json || value_json === "") return "Unknown";
    try {
        // Try parsing as JSON (e.g., for "\"High\"")
        const parsed = JSON.parse(value_json);
        return String(parsed); // Ensure result is string
    } catch(e) {
        // If parsing fails, treat it as a plain string value
        return String(value_json);
    }
};

// Map incident IDs to their attributes
const incidentAttributes = new Map(); // Map<incident_id, { harmLevel?: string, sector?: string }>
const allIncidentIds = new Set();

// Iterate through each classification item returned by the query
for (const classification of allClassificationData) {
    // Find relevant attributes within this classification item
    let currentHarmLevel = null;
    let currentSector = null;

    // Use optional chaining ?. in case attributes array is missing/null
    (classification?.attributes)?.forEach(attr => {
        // Skip if attr is nullish
        if (!attr) return;

        if (attr.short_name === "AI Harm Level") {
            currentHarmLevel = parseValueJson(attr.value_json);
        } else if (attr.short_name === "Sector of Deployment") {
            currentSector = parseValueJson(attr.value_json);
        }
    });

    // If this classification item defined either attribute, apply to its incidents
    if (currentHarmLevel !== null || currentSector !== null) {
        // Use optional chaining ?. in case incidents array is missing/null
        (classification?.incidents)?.forEach(incident => {
            // Check incident and incident_id are not null/undefined
            if (incident?.incident_id != null) {
                const incidentId = incident.incident_id;
                allIncidentIds.add(incidentId);

                // Get existing data or initialize; use nullish coalescing ??
                const existingData = incidentAttributes.get(incidentId) ?? {};

                // Update with values from this classification item (last one seen wins)
                if (currentHarmLevel !== null) {
                    existingData.harmLevel = currentHarmLevel;
                }
                if (currentSector !== null) {
                    existingData.sector = currentSector;
                }
                incidentAttributes.set(incidentId, existingData);
            }
        });
    }
}

console.log(`Found ${allIncidentIds.size} unique incidents linked to CSET classifications.`);

// Group by the combined key "harmLevel|sector" and count
const groupedCounts = new Map(); // Map<"harmLevel|sector", count>

allIncidentIds.forEach(incidentId => {
    // Use nullish coalescing ?? for defaults
    const attrs = incidentAttributes.get(incidentId) ?? {};
    const harmLevel = attrs.harmLevel ?? "Unknown";
    const sector = attrs.sector ?? "Unknown";
    const key = `${harmLevel}|${sector}`;
    // Use nullish coalescing ?? for the count initialization
    groupedCounts.set(key, (groupedCounts.get(key) ?? 0) + 1);
});

// Convert the grouped map into the final array format for plotting
// This uses 'const' and relies on cell scope sharing
heatmapData = Array.from(groupedCounts.entries()).map(([key, count]) => {
    const [harm_level, sector] = key.split('|');
    return { harm_level, sector, count };
});

console.log(`Processed into ${heatmapData.length} unique harm/sector combinations.`);
// Log sample for verification
console.log("Sample Processed Data:", heatmapData.slice(0, 10));

// 'heatmapData' is now defined in this cell's scope and should be
// accessible by the next cell if the notebook environment allows it.

Processing 214 CSET classification items...
Found 214 unique incidents linked to CSET classifications.
Processed into 81 unique harm/sector combinations.
Sample Processed Data: [
  { harm_level: "", sector: "", count: 12 },
  {
    harm_level: "none",
    sector: "administrative and support service activities,human health and social work activities",
    count: 1
  },
  {
    harm_level: "AI tangible harm event",
    sector: "human health and social work activities",
    count: 3
  },
  {
    harm_level: "none",
    sector: "information and communication",
    count: 36
  },
  {
    harm_level: "none",
    sector: "administrative and support service activities",
    count: 3
  },
  {
    harm_level: "none",
    sector: "professional, scientific and technical activities",
    count: 3
  },
  {
    harm_level: "none",
    sector: "Arts, entertainment and recreation,information and communication",
    count: 12
  },
  {
    harm_level: "AI tangible harm event",
    sector: "transportation

In [4]:
// === Plotting Cell: Harm Level vs Sector Heatmap ===

// Ensure Plot and document are imported in a previous cell:
// import * as Plot from "[https://esm.sh/@observablehq/plot";](https://esm.sh/@observablehq/plot";)
// import { document } from "jsr:@manzt/jupyter-helper";

// Assumes 'heatmapData' is defined and populated from the previous cell.
console.log(`Generating heatmap from ${heatmapData.length} data points...`);

// --- Define the Plot ---
const heatmapPlot = Plot.plot({
  // === Configuration ===
  title: "Incident Count by AI Harm Level and Sector of Deployment",
  // Rotate x-axis labels for better readability if sector names are long
  x: { label: "Sector of Deployment", labelAnchor: "center", tickRotate: -60, labelOffset: 85 },
  y: { label: "AI Harm Level" },
  // Configure the color scale and add a legend
  color: {
    scheme: "viridis",
    type: "log",
    // legend: true, // Removed for testing legend rendering issue
    label: "Number of Incidents (log scale)",
    nice: true,
  },

  // Improve layout spacing
  marginTop: 50,
  marginRight: 50,
  marginBottom: 100, // Increased margin for rotated labels
  marginLeft: 150,  // Increased margin for potentially long harm level labels

  // === Marks ===
  marks: [
    // 1. The Heatmap Cells
    Plot.cell(heatmapData, {
      x: "sector",      // Map 'sector' column to the x-axis
      y: "harm_level",  // Map 'harm_level' column to the y-axis
      fill: "count",    // Color the cells based on the 'count' column
      // Add inset to slightly separate cells visually (optional)
      // inset: 0.5,
      title: (d) => `Sector: ${d.sector}\nHarm Level: ${d.harm_level}\nCount: ${d.count}`, // Basic title for cell itself
    }),

    // 2. Text Labels on Cells (Optional but helpful)
    // Plot.text(heatmapData, {
    //   x: "sector",
    //   y: "harm_level",
    //   text: (d) => (d.count > 0 ? d.count : ""), // Show count, hide if 0
    //   // Dynamically set text color based on cell color for contrast
    //   // This requires calculating max count; simpler to just pick one color or omit text fill styling
    //   // fill: (d) => d.count > (maxCount / 2) ? "white" : "black", // Example dynamic fill
    //   fill: "black", // Use black text (adjust if your color scheme is dark)
    //   stroke: "white", // Add white outline for visibility
    //   strokeWidth: 2,
    //   dy: 0, // Center text vertically
    // }),

    // 3. Tooltips for Interactivity (provides details on hover)
    Plot.tip(heatmapData, Plot.pointer({
      x: "sector",
      y: "harm_level",
      title: (d) => `${d.count} incidents\nSector: ${d.sector}\nHarm Level: ${d.harm_level}` // Content of the tooltip
    }))
  ],

  // === Deno/Jupyter Integration ===
  document // Pass the imported document object
}); // End of Plot.plot configuration

// The plot object itself is the implicit return value for rendering
heatmapPlot;

Generating heatmap from 81 data points...


Incident Count by AI Harm Level and Sector of Deployment AI tangible harm event AI tangible harm issue AI tangible harm near-miss Unknown none unclear AI Harm Level Arts, entertainment and recreation Arts, entertainment and recreation,information and communication Arts, entertainment and recreation,professional, scientific and technical activities Education Education,Arts, entertainment and recreation Education,human health and social work activities Education,information and communication Education,law enforcement,public administration Education,professional, scientific and technical activities Education,public administration Unknown accommodation and food service activities accommodation and food service activities,information and communication accommodation and food service activities,transportation and storage accommodation and food service activities,wholesale and retail trade administrative and support service activities administrative and support service activities,Education administrative and support service activities,human health and social work activities administrative and support service activities,information and communication administrative and support service activities,other service activities,financial and insurance activities,Arts, entertainment and recreation,human health and social work activities administrative and support service activities,professional, scientific and technical activities defense financial and insurance activities financial and insurance activities,other human health and social work activities human health and social work activities,public administration information and communication information and communication,Arts, entertainment and recreation information and communication,Education information and communication,law enforcement information and communication,professional, scientific and technical activities information and communication,unclear law enforcement law enforcement,information and communication law enforcement,public administration manufacturing other professional, scientific and technical activities professional, scientific and technical activities,information and communication public administration public administration,financial and insurance activities public administration,human health and social work activities real estate activities real estate activities,public administration transportation and storage transportation and storage,information and communication transportation and storage,wholesale and retail trade wholesale and retail trade wholesale and retail trade,administrative and support service activities wholesale and retail trade,information and communication wholesale and retail trade,transportation and storage Sector of Deployment Sector: 
Harm Level: 
Count: 12 Sector: administrative and support service activities,human health and social work activities
Harm Level: none
Count: 1 Sector: human health and social work activities
Harm Level: AI tangible harm event
Count: 3 Sector: information and communication
Harm Level: none
Count: 36 Sector: administrative and support service activities
Harm Level: none
Count: 3 Sector: professional, scientific and technical activities
Harm Level: none
Count: 3 Sector: Arts, entertainment and recreation,information and communication
Harm Level: none
Count: 12 Sector: transportation and storage
Harm Level: AI tangible harm event
Count: 13 Sector: wholesale and retail trade
Harm Level: none
Count: 7 Sector: manufacturing
Harm Level: none
Count: 3 Sector: transportation and storage
Harm Level: none
Count: 4 Sector: transportation and storage
Harm Level: unclear
Count: 1 Sector: Education
Harm Level: none
Count: 6 Sector: law enforcement,public administration
Harm Level: AI tangible harm event
Count: 3 Sector: law enforcement
Harm Level: AI tangible harm near-miss
Count: 1 Sector: information and communication,Arts, entertainment and recreation
Harm Level: none
Count: 8 Sector: accommodation and food service activities
Harm Lev